In [17]:
# import dependencies
from IPython.display import display, Javascript, Image
from google.colab.output import eval_js
from google.colab.patches import cv2_imshow
from base64 import b64decode, b64encode
import cv2
import numpy as np
import PIL
import io
import html
import time
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# clone darknet repo
!git clone https://github.com/AlexeyAB/darknet

Cloning into 'darknet'...
remote: Enumerating objects: 15283, done.
remote: Counting objects: 100% (97/97), done.
remote: Compressing objects: 100% (68/68), done.
remote: Total 15283 (delta 40), reused 50 (delta 26), pack-reused 15186
Receiving objects: 100% (15283/15283), 13.75 MiB | 17.31 MiB/s, done.
Resolving deltas: 100% (10349/10349), done.


In [ ]:
# change makefile to have GPU, OPENCV and LIBSO enabled
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!sed -i 's/CUDNN_HALF=0/CUDNN_HALF=1/' Makefile
!sed -i 's/LIBSO=0/LIBSO=1/' Makefile
!pip install flask-ngrok
!make

In [ ]:
!cd /content/darknet

In [ ]:
# get bthe scaled yolov4 weights file that is pre-trained to detect 80 classes (objects) from shared google drive
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1V3vsIaxAlGWvK4Aar9bAiK5U0QFttKwq' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1V3vsIaxAlGWvK4Aar9bAiK5U0QFttKwq" -O yolov4-csp.weights && rm -rf /tmp/cookies.txt

In [19]:
# import darknet functions to perform object detections
from darknet.darknet import *
#from darknet import *
# load in our YOLOv4 architecture network
network, class_names, class_colors = load_network("/content/drive/MyDrive/yolo/yolo4-obj.cfg", "/content/drive/MyDrive/yolo/coco.data", "/content/drive/MyDrive/yolo/yolo4-obj_last.weights")
width = network_width(network)
height = network_height(network)

# darknet helper function to run detection on image
def darknet_helper(img, width, height):
  darknet_image = make_image(width, height, 3)
  img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
  img_resized = cv2.resize(img_rgb, (width, height),
                              interpolation=cv2.INTER_LINEAR)

  # get image ratios to convert bounding boxes to proper size
  img_height, img_width, _ = img.shape
  width_ratio = img_width/width
  height_ratio = img_height/height

  # run model on darknet style image to get detections
  copy_image_from_bytes(darknet_image, img_resized.tobytes())
  detections = detect_image(network, class_names, darknet_image)
  free_image(darknet_image)
  return detections, width_ratio, height_ratio

# run test on person.jpg image that comes with repository
#image = cv2.imread("/content/drive/MyDrive/yolo/KakaoTalk_20210705_041827454.jpg")
def labeling(image):
  label = []
  labels = []
  detections, width_ratio, height_ratio = darknet_helper(image, width, height)

  for label, confidence, bbox in detections:
    left, top, right, bottom = bbox2points(bbox)
    left, top, right, bottom = int(left * width_ratio), int(top * height_ratio), int(right * width_ratio), int(bottom * height_ratio)
    cv2.rectangle(image, (left, top), (right, bottom), class_colors[label], 2)
    cv2.putText(image, "{} [{:.2f}]".format(label, float(confidence)),
                      (left, top - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5,
                      class_colors[label], 2)
    label = [left, top, right, bottom]
    labels.append(label)
  return dict(zip(range(len(labels)),labels))
  #cv2_imshow(image)
def draw(detection, imgname):
  g_color = (0,255,0)
  img = cv2.imread(imgname,cv2.imread_COLOR)


In [ ]:
#result = {'0': [275, 127, 358, 212, 'umook', 1.0], '1': [264, 46, 320, 110, 'gaeranmalee', 0.574], '2': [179, 44, 246, 107, 'kongnamul', 0.99], '3': [78, 223, 174, 319, 'rice', 1.0], '4': [102, 52, 162, 102, 'kimchigeon', 1.0], '5': [113, 116, 228, 213, 'kimchigeon', 1.0], '6': [356, 38, 446, 136, 'kkatip', 0.839], '7': [193, 211, 299, 329, 'mookoook', 0.48]}
#src = cv2.imread("/content/some_image.jpg",cv2.IMREAD_COLOR)

def drawing(result,src):
  color = (0,255,0)
  tmp = src.copy()
  print(type(result))
  for key,value in result.items():
    print(key,value)
    left, top, right, bottom = value[0],value[1],value[2],value[3]
    cv2.rectangle(tmp, (left, top), (right, bottom), color, 2)
    cv2.putText(tmp, "{} [{:.2f}]".format(value[4], float(value[5])),
                        (left, top - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5,
                        color, 2)
  filename = '/content/some_image_finaltest.jpg'
  cv2.imwrite(filename, tmp)



In [20]:
import tensorflow
from PIL import Image, ImageOps
import numpy as np
import cv2

def predict_detail(filename):
    # Disable scientific notation for clarity
    np.set_printoptions(suppress=True)

    # Load the model
    model = tensorflow.keras.models.load_model('/content/drive/MyDrive/yolo/keras_model_18.h5')

    # Create the array of the right shape to feed into the keras model
    # The 'length' or number of images you can put into the array is
    # determined by the first position in the shape tuple, in this case 1.
    data = np.ndarray(shape=(1, 224, 224, 3), dtype=np.float32)

    # Replace this with the path to your image
    image = Image.open(filename)

    #resize the image to a 224x224 with the same strategy as in TM2:
    #resizing the image to be at least 224x224 and then cropping from the center
    size = (224, 224)
    image = ImageOps.fit(image, size, Image.ANTIALIAS)

    #turn the image into a numpy array
    image_array = np.asarray(image)

    # display the resized image
    image.show()

    # Normalize the image
    normalized_image_array = (image_array.astype(np.float32) / 127.0) - 1

    # Load the image into the array
    data[0] = normalized_image_array

    # run the inference
    prediction = model.predict(data)
    print('#####',type(prediction))
    return prediction.tolist()
     


foods = ['dontgas','gaeranmalee','gamjache','jaeukbookum','kimchigeon','kimchi_baechu','kkakdugi','kkatip','kongnamul','mookoook','rice','sausagebokum','ugugikook','umook']

def detectfood(xylist):

  def f1(x):
    return dictionary[x]

  src = cv2.imread("/content/some_image.jpg",cv2.IMREAD_COLOR)
  x,x_d,y,y_d = xylist[0],xylist[2],xylist[1],xylist[3]
  if x < 0:
    x = 1
  if x_d < 0:
    x_d = 1
  if y < 0:
    y = 1
  if y_d < 0:
    y_d = 1
  print(x,x_d,y,y_d)
  dst = src.copy()
  roi = src[y:y_d,x:x_d].copy()
  filename = '/content/some_image_0.jpg'  # I assume you have a way of picking unique filenames
  cv2.imwrite(filename, roi)


  predict_list_all= predict_detail(filename)
  predict_list = predict_list_all[0]
  print('########predictlist###',predict_list)
  for i in range(len(predict_list)):
    predict_list[i] = round(predict_list[i],3)

  print(predict_list)
  #predict_list=predict_str.split(' ')
  dictionary = dict(zip(foods, predict_list))
  most = max(dictionary.keys(),key=f1)

  return most,dictionary[most]




In [21]:
import json
from flask import Flask ,request,jsonify
from flask_ngrok import run_with_ngrok
import base64
import cv2
app = Flask(__name__)
run_with_ngrok(app)   
  
@app.route("/")
def home():
    return "<h1>GFG is great platform to learn</h1>"
    


@app.route("/convert", methods=['POST'])
def converter():
    data = request.get_json()
    data = json.loads(data)
    print('###############', data)
    print('###############', request.is_json)
    if 'image' not in data:
        return "", 400
    img_data = base64.b64decode(data["image"])
    
    
    filename = '/content/some_image.jpg'  # I assume you have a way of picking unique filenames
    with open(filename, 'wb') as f:
        f.write(img_data)
    image = cv2.imread("/content/some_image.jpg")
    detection = labeling(image)
    print('###############',detection)
    print(type(detection))
    for i in range(len(detection)):
      key,value = detectfood(detection[i])
      detection[i].append(key)
      detection[i].append(value)

    return detection, 200


app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://a31ec7fb36e7.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [22/Jul/2021 07:40:54] "GET / HTTP/1.1" 200 -


############### {'image': '...'}
############### True
############### {0: [38, 633, 571, 1046], 1: [557, 129, 816, 428], 2: [730, 438, 1405, 1022], 3: [-6, 173, 349, 449], 4: [356, 135, 612, 428], 5: [855, 59, 1305, 360]}
<class 'dict'>
38 571 633 1046
##### <class 'numpy.ndarray'>
########predictlist### [2.4001150450203568e-05, 2.8783422749256715e-05, 0.0002563484595157206, 6.839010069370488e-08, 3.613099863741809e-08, 6.450833467397388e-08, 5.422473350336077e-06, 1.4399404335563304e-07, 0.0004000260669272393, 5.484740904648788e-05, 0.9992272853851318, 1.1967811985869048e-07, 1.873947397257325e-08, 2.860430868167896e-06]
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.999, 0.0, 0.0, 0.0]
557 816 129 428
##### <class 'numpy.ndarray'>
########predictlist### [0.999839186668396, 1.7476901120971888e-05, 3.0255861815930984e-07, 8.857372790771478e-07, 3.764234861591831e-06, 4.904260549665196e-06, 1.9103320028079906e-06, 1.9182870403255947e-07, 1.6956323012351504e-09, 2.6051718577946303e

127.0.0.1 - - [22/Jul/2021 07:41:29] "POST /convert HTTP/1.1" 200 -


##### <class 'numpy.ndarray'>
########predictlist### [1.1908045962627511e-06, 2.8010264941258356e-05, 0.0013254312798380852, 0.13780249655246735, 0.0007996296626515687, 7.03374496424658e-07, 1.4613055100198835e-05, 1.4878450826927292e-07, 0.033726245164871216, 2.584191349797038e-07, 6.185101938172011e-06, 0.0008080708212219179, 0.0003691388410516083, 0.825117826461792]
[0.0, 0.0, 0.001, 0.138, 0.001, 0.0, 0.0, 0.0, 0.034, 0.0, 0.0, 0.001, 0.0, 0.825]
############### {'image': '...'}
############### True
############### {0: [497, 26, 586, 124], 1: [277, -3, 429, 133], 2: [649, 51, 783, 211], 3: [140, 17, 245, 114], 4: [645, 294, 881, 468], 5: [97, 156, 252, 284], 6: [202, 340, 404, 521], 7: [442, 153, 647, 377]}
<class 'dict'>
497 586 26 124
##### <class 'numpy.ndarray'>
########predictlist### [6.32976480119396e-07, 0.9478291273117065, 8.261301365219964e-11, 8.849608988755308e-17, 4.7842468120506965e-06, 3.1901668080536183e-06, 0.05213959887623787, 7.459052819847045e-11, 4.0976715354190

127.0.0.1 - - [22/Jul/2021 07:42:48] "POST /convert HTTP/1.1" 200 -


##### <class 'numpy.ndarray'>
########predictlist### [6.0701381698891055e-06, 5.233611659605231e-07, 2.840047272911761e-06, 4.6756550631243954e-08, 3.2669607463731154e-08, 1.6284921855458379e-07, 0.0005672554252669215, 3.506321633039988e-08, 4.491604954637296e-08, 4.403765387905878e-07, 2.791220943265671e-08, 2.8674274290096946e-05, 7.498419290641323e-05, 0.9993189573287964]
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.001, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.999]
